In [3]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import ipywidgets as ipw


In [4]:
xi = np.linspace(-9, 9, 13)
f1 = lambda x : (2*x**4 + np.sin(2*x) + np.random.randint(-4, 4))/ np.sqrt(2*x**4 + 7)
f2 = lambda x : (2*x**3 )/np.sqrt(3*x**2 + 1) + np.sin(2*x)  + np.random.randint(-9, 9)
f3 = lambda x: sum(np.random.randint(-9, 9)*x**i for i in range(0, 6)) + np.random.randint(-60, 60)
y1 = f1(xi)
y2 = f2(xi)
y3 = f3(xi)

px.scatter(x=xi, y=y3)

# Programmez l'approximation polynomiale d'un ensemble de points par la méthodes des moindres carrés


In [6]:

def moindre_carre(x, y, order):
    X = np.concatenate([[x**i] for i in range(order+1)], axis=0).T
    return np.linalg.inv(X.T@X)@X.T@y

def estimate(x, theta, order):
    assert len(theta) == order+1
    x_puissance = np.concatenate([[x**i] for i in range(order+1)], axis=0).T
    return x_puissance@theta 
    

moindre_carre(xi, y1, order=1)


array([ 4.41734684e+01, -4.74079175e-05])

## estimation des erreurs


In [7]:
def MAE(y, y_pred ):
    return np.mean(np.abs(y-y_pred))

def MSE(y, y_pred ):
    return np.mean((y-y_pred)**2)


## Visualisation

In [9]:
x = np.linspace(-9, 9, 100)

orders = list(range(1, 6))
thetas = [moindre_carre(xi, y2, order=i) for i in orders]
y_pred = [estimate(x, thetas[i], order=order) for i, order in enumerate(orders)]


data = pd.DataFrame(columns=["xi", "yi", "order"])

for i, order in enumerate(orders):
    df = pd.DataFrame({'xi': x, 'yi': y_pred[i]})
    df = df.assign(order=order)
    data = pd.concat([data, df])

fig = px.line(data, x="xi", y="yi", color="order")

fig.add_trace(go.Scatter(x=xi, y=y2, mode='markers', name='points'))
fig.show()

In [20]:


def visualise(f, order_max, pts_inter, etendue):
    f = function[f]
    max_val = etendue // 2
    xi = np.linspace(-max_val, max_val, pts_inter)
    y = f(xi)
    x = np.linspace(-max_val, max_val, 200)

    orders = list(range(1, order_max+1))
    thetas = [moindre_carre(xi, f(xi), order=i) for i in orders]
    y_true = f(x)
    y_pred = [estimate(x, thetas[i], order=orders[i]) for i, _ in enumerate(orders)]

    data = pd.DataFrame(columns=["xi", "yi", "order"])

    for i, _ in enumerate(orders):
        df = pd.DataFrame({'xi': x, 'yi': y_pred[i]})
        df = df.assign(order=orders[i])
        df = df.assign(MAE=MAE(y_true, y_pred[i]))
        df = df.assign(MSE=MSE(y_true, y_pred[i]))
        data = pd.concat([data, df])

    print(data.columns)
    fig = px.line(data, x="xi", y="yi", color="order", hover_data={'MAE':True, "MSE": True})

    fig.add_trace(go.Scatter(x=xi, y=y, mode='markers', name='points'))
    return fig


function = {"f1": f1, "f2": f2, "f3": f3}
orders = range(2, 15)
nbre_pts_interpol = range(10, 20)

etendue = range(25, 100, 5)

ipw.interact(visualise, f=function.keys(), order_max=orders, pts_inter=nbre_pts_interpol, etendue=etendue)

interactive(children=(Dropdown(description='f', options=('f1', 'f2', 'f3'), value='f1'), Dropdown(description=…

<function __main__.visualise(f, order_max, pts_inter, etendue)>